# Bond Dimension Required vs. Number of Nodes

In [1]:
include("../funcs/adjacency.jl")  
include("../funcs/mps.jl")  
include("../funcs/hamiltonian.jl")
include("../funcs/json.jl")

write_data_as_json (generic function with 1 method)

## Params

In [ ]:
num_graphs_to_avg = 10 # number of graphs to average over for each (N, σ) pair
N_vals = 10:2:100 # number of nodes to test
σ_vals = [0.0, 0.001, 0.002] # sigma values to test for the adjacency martrix

J = 1.0 # interaction strength
Δ = 1.5 # anisotropy parameter




# Generate MPOs

In [ ]:

data = load_data_from_json("../data/bond_dim_vs_nodes_data.json")
display(data)

In [ ]:
using ITensorMPS, ITensors
using ProgressMeter
import Statistics

if data == nothing
    data = Dict() # {σ => {N => (max_bond_dim, error)}}

    total_iterations = length(N_vals) * length(σ_vals)
    progress = Progress(total_iterations, desc="Calculating bond dimensions")

    for N in N_vals
        for σ in σ_vals
            local_averaging = σ != 0.0 ? num_graphs_to_avg : 1 # only bother averaging if we actually have randomness

            bond_dims = Int[]
            for _ in 1:local_averaging
                wam = generate_fully_connected_wam(N, σ) # weighted adjacency matrix
                ψ_mps, sites = create_MPS(N)
                H = create_xxz_hamiltonian_mpo(N, wam, J, Δ, sites)
                max_bond_dim = maxlinkdim(H) # maximum bond dimension of the MPO hamiltonian
                push!(bond_dims, max_bond_dim)
            end

            avg_bond_dim = Statistics.mean(bond_dims) # average over local_averaging graphs
            error = σ != 0.0 ? Statistics.std(bond_dims) : 0 # standard deviation if we have randomness, else 0

            # create the sigma key if it doesn't exist
            if !haskey(data, σ)
                data[σ] = Dict()
            end
            data[σ][N] = (avg_bond_dim, error)

            next!(progress)
        end
    end
end

In [ ]:
# save the data in json format
write_data_as_json("../data/bond_dim_vs_nodes_data.json", data)

In [ ]:
# plot the data using plots
using Plots

σ_to_plot = [0.0, 0.002]

# we should have all the sigma values on the same plot, we want avg_bond_dim vs N with error bars
colors = [:red, :blue, :green, :orange, :purple]
plt = plot(title="Maximum Bond Dimension vs Number of Nodes \n (averaged over $num_graphs_to_avg graphs)",
    xlabel="Number of Nodes (N)",
    ylabel="Maximum Bond Dimension",
    legend=:topleft)

for (i, σ) in enumerate(keys(data))
    if !(σ in σ_to_plot)
        continue
    end
    σ_data = data[σ]
    N_vals = collect(keys(σ_data))
    avg_bond_dims = [σ_data[N][1] for N in N_vals]
    errors = [σ_data[N][2] for N in N_vals]

    plot!(plt, N_vals, avg_bond_dims, 
          yerror=errors,
          label="σ = $σ", 
          color=colors[i % length(colors) + 1],
          marker=:circle,
          markersize=3,
          linewidth=2)
end
savefig(plt, "../assets/graphs/bond_dim_req_vs_nodes_ham_mpo.png")
display(plt)

# Same thing, but for Ground State MPS at sigma=0.0

In [2]:
# new parameters
N_vals = 10:1:100 # number of nodes to test
σ_vals = [0.0, 0.001, 0.002] # sigma values to test for the adjacency martrix

num_graphs_to_avg = 5 # number of graphs to average over for each (N, σ) pair
max_bond_dim = 1000 # maximum bond dimension for the MPS

# Hamiltonian parameters - Heisenberg XXZ
J = -0.5 # interaction strength
Δ = 0.5 # anisotropy parameter

RERUN = true

true

In [ ]:
# attempt to load data first 
using JLD2
data_gs_mps = load("../data/bond_dim_vs_nodes_gs_mps_avg_$num_graphs_to_avg.jld2", "data_gs_mps")

Dict{Any, Any} with 2 entries:
  0.0   => Dict{Any, Any}(56=>(24.0, 0), 35=>(18.0, 0), 55=>(23.0, 0), 16=>(9.0…
  0.002 => Dict{Any, Any}(56=>(24.0, NaN), 35=>(18.0, NaN), 55=>(23.0, NaN), 16…

In [4]:
using Pkg
println("Active project: ", Pkg.project().path)
println("Julia threads: ", Threads.nthreads())
println("CPU cores: ", Sys.CPU_THREADS)

Active project: /Users/jamesneville-rolfe/Library/CloudStorage/GoogleDrive-james.nevillerolfe@gmail.com/My Drive/Documents/University/fourth_year/MSci-Project/Project.toml
Julia threads: 8
CPU cores: 8


In [ ]:

using Base.Threads
import Statistics
using ITensorMPS, ITensors
using ProgressMeter

if data_gs_mps == nothing || RERUN


    println("Using $(Threads.nthreads()) threads for parallel computation.")
    N_results = Vector{Any}(undef, length(N_vals))

    completed = Threads.Atomic{Int}(0)
    total_tasks = length(N_vals) * length(σ_vals)

    total_iterations = length(N_vals) * length(σ_vals)

    @threads for n_idx in 1:length(N_vals)
        println("Thread $(Threads.threadid()) starting N index $n_idx")
        N = N_vals[n_idx]
        N_data = Dict()
        
        thread_id = Threads.threadid()
        progress = Progress(length(σ_vals), desc="Thread $thread_id: Calculating bond dimensions for N=$N")

        for σ in σ_vals
            local_averaging = σ != 0.0 ? num_graphs_to_avg : 1
            bond_dims = Int[]
            
            for _ in 1:local_averaging
                wam = generate_fully_connected_wam(N, σ)
                ψ_mps, sites = create_MPS(N)
                H = create_xxz_hamiltonian_mpo(N, wam, J, Δ, sites)
                energy, ψ_gs = solve_xxz_hamiltonian_dmrg(H, ψ_mps, 30, max_bond_dim, 1e-10)
                bond_dim = maxlinkdim(ψ_gs)
                push!(bond_dims, bond_dim)
            end
            
            avg_bond_dim = Statistics.mean(bond_dims)
            # standard error on the mean
            if σ != 0.0
                error = Statistics.std(bond_dims) / sqrt(length(bond_dims)) # std/sqrt(sample size)
            else
                error = 0.0
            end
            N_data[σ] = (avg_bond_dim, error)
            next!(progress)
        end
        
        Threads.atomic_add!(completed, length(σ_vals))

        N_results[n_idx] = (N, N_data)
        println("Completed N = $N on thread $(Threads.threadid()). Progress: $(completed[]) / $total_tasks")
    end
    
    data_gs_mps = Dict() # {σ => {N => (max_bond_dim, error)}}
    # Combine results
    for (N, N_data) in N_results
        for (σ, values) in N_data
            if !haskey(data_gs_mps, σ)
                data_gs_mps[σ] = Dict()
            end
            data_gs_mps[σ][N] = values
        end
    end
end

Using 8 threads for parallel computation.
Thread 1 starting N index 25
Thread 8 starting N index 59
Thread 2 starting N index 1
Thread 7 starting N index 48
Thread 5 starting N index 37
Thread 6 starting N index 13
Thread 4 starting N index 81
Thread 3 starting N index 70
Completed N = 10 on thread 3. Progress: 3 / 273
Thread 3 starting N index 2


Thread 3: Calculating bond dimensions for N=10 100%|█████| Time: 0:01:24
Thread 3: Calculating bond dimensions for N=11 100%|█████| Time: 0:00:57


Completed N = 11 on thread 3. Progress: 6 / 273
Thread 3 starting N index 3


Thread 3: Calculating bond dimensions for N=12  67%|███▍ |  ETA: 0:00:14

In [ ]:
using JLD2

# Save data
jldsave("../data/bond_dim_vs_nodes_gs_mps.jld2"; data_gs_mps)

In [ ]:
# then plot the data
using Plots
σ_to_plot = [0.0, 0.002]
colors = [:red, :blue, :green, :orange, :purple]
plt = plot(title="Maximum Bond Dimension of Ground State MPS\nvs Number of Nodes \n (averaged over $num_graphs_to_avg graphs)",
    xlabel="Number of Nodes (N)",
    ylabel="Maximum Bond Dimension",
    legend=:topright)

for (i, σ) in enumerate(keys(data_gs_mps))
    if !(σ in σ_to_plot)
        continue
    end
    σ_data = data_gs_mps[σ]
    N_vals = collect(keys(σ_data))
    N_vals_sorted = sort(N_vals)
    avg_bond_dims = [σ_data[N][1] for N in N_vals_sorted]
    errors = [σ_data[N][2] for N in N_vals_sorted] # should be 0 for σ=0

    plot!(plt, N_vals_sorted, avg_bond_dims,
          yerror=errors,
          label="σ = $σ", 
          color=colors[i % length(colors) + 1],
          marker=:circle,
          markersize=3,
          linewidth=2)
end
savefig(plt, "../assets/graphs/bond_dim_req_vs_nodes_gs_mps_avg_$(num_graphs_to_avg).png")
display(plt)    